# Python Foundations → Professional Practice (3-hour workshop)

This notebook is designed for live teaching.  
It follows a logical progression:

1. **Python mental model** (names, objects, mutability)
2. **Core data types + truthiness**
3. **Comprehensions & generators**
4. **Functions & type hints**
5. **Robustness** (errors, file I/O)
6. **Organization** (modules, packaging overview)
7. **OOP basics**
8. **Performance awareness**
9. **Pitfalls & clean coding**
10. **Tooling** (venv + logging + reproducibility)

> Tip: Run cells top-to-bottom. Encourage students to modify values and re-run.

## Setup (optional)
Run this if you want a couple helper imports for demos.

In [ ]:
from __future__ import annotations

from dataclasses import dataclass
from typing import Iterable, Iterator, Optional

# Hour 1 — Python Foundations & Mental Model

## 1) Python mental model: names → objects (references)

In Python, variables are **names bound to objects**.  
Assignment does **not** copy the object—it points another name to the same object (for mutable objects).

Let's see how this creates surprising behavior with lists.

In [ ]:
a = [1, 2, 3]
b = a          # b points to the same list object
b.append(4)

print("a:", a)
print("b:", b)
print("a is b:", a is b)

### Copying: shallow vs deep
- `list(a)` or `a.copy()` makes a *shallow* copy (copies the container, not nested objects).
- For nested structures, consider `copy.deepcopy`.

In [ ]:
import copy

x = [[1, 2], [3, 4]]
y = x.copy()           # shallow copy
z = copy.deepcopy(x)   # deep copy

y[0].append(99)        # modifies the nested list shared with x
z[1].append(88)        # modifies only z

print("x:", x)
print("y:", y)
print("z:", z)

## 2) Mutability (why it matters)

- **Immutable**: `int`, `float`, `bool`, `str`, `tuple`, `frozenset`
- **Mutable**: `list`, `dict`, `set`, most user-defined objects

Immutable objects can't be changed "in place"; operations create new objects.

In [ ]:
s = "hi"
t = s
s += "!"  # creates a new string object

print("s:", s)
print("t:", t)
print("s is t:", s is t)  # usually False after +=

## 3) Core data types + truthiness

Truthiness is what `if x:` uses.
Common falsy values:
- `False`, `None`, `0`, `0.0`
- `""`, `[]`, `{}`, `set()`

In [ ]:
values = [False, None, 0, 1, "", "x", [], [1], {}, {"a": 1}, set(), {1}]

for v in values:
    print(f"{v!r:>10} -> {bool(v)}")

## 4) f-strings (use these)
They’re readable and fast enough for most use cases.

In [ ]:
name = "Mia"
score = 93.4567
print(f"Student {name} scored {score:.1f}.")
print(f"Debug view: {score=}, {name=}")

## 5) Comprehensions & generators

- List comprehension: builds the whole list in memory.
- Generator expression: produces values lazily (often better for large data).

In [ ]:
nums = range(10)

squares_list = [n * n for n in nums if n % 2 == 0]
squares_gen  = (n * n for n in nums if n % 2 == 0)

print("list:", squares_list)
print("gen:", squares_gen)  # generator object
print("consume gen:", list(squares_gen))

### Mini exercise
1) Create a list of strings `"item-0" ... "item-9"` using a comprehension.  
2) Create a generator that yields numbers divisible by 3 from 0..99 and sum them.

In [ ]:
# Exercise 1
items = [f"item-{i}" for i in range(10)]
print(items)

# Exercise 2
g = (n for n in range(100) if n % 3 == 0)
print(sum(g))

## 6) Functions + type hints

Guidelines:
- Prefer small, testable functions
- Avoid globals
- Add type hints when it clarifies intent

In [ ]:
def mean(values: list[float]) -> float:
    if not values:
        raise ValueError("values must be non-empty")
    return sum(values) / len(values)

print(mean([1.0, 2.0, 3.0]))

## 7) Pitfall: mutable default argument bug

Default arguments are evaluated **once** at function definition time, not each call.

In [ ]:
def add_item_bad(x: int, items: list[int] = []):
    items.append(x)
    return items

print(add_item_bad(1))
print(add_item_bad(2))  # surprises people

Fix it with `None` and create a new list each call.

In [ ]:
def add_item_good(x: int, items: Optional[list[int]] = None) -> list[int]:
    if items is None:
        items = []
    items.append(x)
    return items

print(add_item_good(1))
print(add_item_good(2))

# Hour 2 — Writing Real Code (robust + organized)

## 8) Error handling patterns

Use exceptions to handle *expected* error cases.  
Don't swallow exceptions silently.

In [ ]:
def parse_int(s: str) -> int:
    try:
        return int(s)
    except ValueError as e:
        raise ValueError(f"Not an integer: {s!r}") from e

for text in ["10", "003", "oops"]:
    try:
        print(text, "->", parse_int(text))
    except ValueError as err:
        print("error:", err)

## 9) File I/O

Use `with open(...)` so files close reliably.

In [ ]:
from pathlib import Path

p = Path("demo_notes.txt")
p.write_text("line1\nline2\n", encoding="utf-8")

with p.open("r", encoding="utf-8") as f:
    for line in f:
        print("read:", line.strip())

# Cleanup (optional)
# p.unlink()

### JSON example (common in data work)

In [ ]:
import json
from pathlib import Path

data = {"course": "Python", "students": [{"name": "A", "score": 90}, {"name": "B", "score": 85}]}

Path("demo.json").write_text(json.dumps(data, indent=2), encoding="utf-8")

loaded = json.loads(Path("demo.json").read_text(encoding="utf-8"))
print(loaded["students"][0])

## 10) Modules & code organization (conceptual)

In real projects, avoid one giant notebook/script. Structure code like:

- `project/`
  - `src/project_name/`
    - `__init__.py`
    - `core.py`
    - `io.py`
  - `tests/`
  - `pyproject.toml`

In a notebook, we simulate this by writing a small module file and importing it.

In [ ]:
# Write a tiny module file on the fly
from pathlib import Path

Path("workshop_utils.py").write_text(
"""def shout(msg: str) -> str:
    return msg.upper() + "!!!"
""",
encoding="utf-8"
)

import workshop_utils
print(workshop_utils.shout("hello module"))

## 11) Packaging overview (conceptual)

Modern Python packaging often uses **pyproject.toml** (PEP 517/518).  
For class, focus on:
- Keep dependencies pinned (reproducibility)
- Keep code modular
- Have a single entry point (script / CLI)

## 12) OOP basics (only what you need)

Use classes when you want to bundle:
- data + behavior
- stateful workflows

Otherwise, functions are often simpler.

In [ ]:
@dataclass
class Counter:
    value: int = 0

    def inc(self, n: int = 1) -> None:
        self.value += n

    def reset(self) -> None:
        self.value = 0

c = Counter()
c.inc()
c.inc(5)
print(c.value)
c.reset()
print(c.value)

# Hour 3 — Performance + Pitfalls + Professional Practice

## 13) Performance demo with `%timeit`

In Jupyter, `%timeit` measures execution time robustly.
Compare list comprehension vs explicit loop.

In [ ]:
# If running in plain Python (not Jupyter), this won't work.
# In Jupyter, run this cell as-is.

import random

data = [random.random() for _ in range(100_000)]

def loop_sum(xs: list[float]) -> float:
    s = 0.0
    for x in xs:
        s += x
    return s

def builtin_sum(xs: list[float]) -> float:
    return sum(xs)

# Jupyter magic:
# %timeit loop_sum(data)
# %timeit builtin_sum(data)

print("Uncomment the %timeit lines above in a live Jupyter environment.")

## 14) Another performance concept: generators save memory

For large ranges, a generator avoids materializing the full list.

In [ ]:
def count_even_squares(n: int) -> int:
    # generator expression: lazy
    return sum(1 for i in range(n) if (i*i) % 2 == 0)

print(count_even_squares(10_000))

## 15) Common pitfalls checklist

- Mutable default args ✅ (we covered)
- Copy vs reference ✅ (we covered)
- Modifying a list while iterating

In [ ]:
nums = [1, 2, 3, 4, 5, 6]

# Bad: modifying while iterating (skips elements)
bad = nums.copy()
for x in bad:
    if x % 2 == 0:
        bad.remove(x)
print("bad:", bad)

# Good: create a new list
good = [x for x in nums if x % 2 != 0]
print("good:", good)

## 16) Clean coding mini refactor

Goal:
- avoid global variables
- write a function
- add type hints
- use f-strings

In [ ]:
# "Before" (intentionally messy)
tax_rate = 0.0925

def total_with_tax(prices):
    t = 0
    for p in prices:
        t += p
    return t * (1 + tax_rate)

print(total_with_tax([10, 20, 30]))

In [ ]:
# "After" (cleaner)
def total_with_tax_clean(prices: list[float], tax_rate: float) -> float:
    subtotal = sum(prices)
    return subtotal * (1 + tax_rate)

prices = [10.0, 20.0, 30.0]
print(f"Total: {total_with_tax_clean(prices, tax_rate=0.0925):.2f}")

## 17) Logging basics (print vs logging)

For serious projects, use `logging` so you can control verbosity and route output.

In [ ]:
import logging

logging.basicConfig(level=logging.INFO, format="%(levelname)s:%(name)s:%(message)s")
logger = logging.getLogger("workshop")

def compute(x: int) -> int:
    logger.info("Computing with x=%s", x)
    return x * x

print(compute(7))

## 18) Virtual environments & reproducibility (notes)

**Create a venv:**
```bash
python -m venv .venv
source .venv/bin/activate   # macOS/Linux
# .venv\Scripts\activate  # Windows
pip install -r requirements.txt
```
**Freeze deps:**
```bash
pip freeze > requirements.txt
```
In modern projects, prefer `pyproject.toml` + a tool like `uv`/`pip-tools`/`poetry`.

# Wrap-up

Key habits:
- Use f-strings
- Write small functions
- Avoid globals
- Use type hints where they reduce ambiguity
- Use venv for reproducibility
- Use logging for real projects
- Keep code modular